# Stanford CME 241 (Winter 2024) - Assignment 3

**Due: Jan 29 @ 11:59pm Pacific Time on Gradescope.**

Assignment instructions:
- **Please solve questions 1 and 2, and choose one of questions 3 or 4.**
- Empty code blocks are for your use. Feel free to create more under each section as needed.

Submission instructions:
- When complete, fill out your publicly available GitHub repo file URL and group members below, then export or print this .ipynb file to PDF and upload the PDF to Gradescope.

*Link to this ipynb file in your public GitHub repo (replace below URL with yours):* 

https://github.com/LilyY24/CME241_TK-XL-JR/blob/main/assignments/assignment3.ipynb

*Group members (replace below names with people in your group):* 
- Tiankai Yan
- Jingruo Sun
- Xiaoli Yang

## Imports

In [ ]:
from typing import Sequence
import numpy as np

from typing import Tuple, Dict, Mapping, Sequence, List, Iterator

from dataclasses import dataclass
from scipy.stats import poisson
import numpy as np

from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.distribution import Categorical

## Question 1
**Analytic Optimal Actions and Cost.** 
Consider a continuous-states, continuous-actions, discrete-time, non-terminating MDP with state space as $\mathbb{R}$ and action space as $\mathbb{R}$. When in state $s\in \mathbb{R}$, upon taking action $a\in \mathbb{R}$, one transitions to next state $s' \in \mathbb{R}$ according to a normal distribution $s' \sim \mathcal{N}(s, \sigma^2)$ for a fixed variance $\sigma^2 \in \mathbb{R}^+$. The corresponding cost associated with this transition is $e^{as'}$, i.e., the cost depends on the action $a$ and the state $s'$ one transitions to. The problem is to minimize the infinite-horizon **Expected Discounted-Sum of Costs** (with discount factor $\gamma < 1$). For this assignment, solve this problem just for the special case of $\gamma = 0$ (i.e., the myopic case) using elementary calculus. Derive an analytic expression for the optimal action in any state and the corresponding optimal cost.


First, we prove the lemma that $\mathbb{E}[e^X] = e^{\mu + \frac{\sigma^2}{2}}$ where $X \sim \mathcal{N}(\mu, \sigma^2)$.

Let $X = \mu + \sigma Z$ where $Z$ is the standard normal variable $\sim \mathcal{N}(0, 1)$. Then we have: 
$\begin{align}
    E[e^X] 
    &= E[e^{\mu+\frac{\sigma^2}{2}Z}] \\
    & = \int_\infty^\infty e^{\mu+\frac{\sigma^2}{2}} \frac{1}{\sqrt{2\pi}} e^{-\frac{z^2}{2}} dz \\
    &= \frac{1}{\sqrt{2\pi}} e^{\mu} \int_{-\infty}^{\infty} e^{\sigma z} e^{-\frac{z^2}{2}} \\
    &= \frac{1}{\sqrt{2\pi}} e^{\mu + \frac{\sigma^2}{2}} \int_{-\infty}^{\infty} e^{-\frac{(z-\sigma)^2}{2}} dz \\
    &= e^{\mu + \frac{\sigma^2}{2}}.
\end{align}$

Then in our problem, given $s' \sim \mathcal{N}(s, \sigma^2)$, we have $E[e^{as'}] = e^{as + \frac{1}{2}\sigma^2 a^2}$. Denote this as $f(a)$.

Since we want to minimize the cost, so we set 
$\begin{align}
    f'(a) = (s + \sigma^2 a) e^{sa + \frac{1}{2}\sigma^2 a^2} = 0,
\end{align}$
which gives us the optimal action $a^* = -\frac{s}{\sigma^2}$.

Therefore, we can also get the optimal cost as $e^{a^* s + \frac{1}{2}\sigma^2 (a^*)^2} = e^{-\frac{s^2}{2\sigma^2}}$.

## Question 2
**Manual Value Iteration.** 
Consider a simple MDP with $\mathcal{S} = \{s_1, s_2, s_3\}, \mathcal{T} =\{s_3\}, \mathcal{A} = \{a_1, a_2\}$. The State Transition Probability function
$$\mathcal{P}: \mathcal{N} \times \mathcal{A} \times \mathcal{S} \rightarrow [0, 1]$$
is defined as:
$$\mathcal{P}(s_1, a_1, s_1) = 0.2, \mathcal{P}(s_1, a_1, s_2) = 0.6, \mathcal{P}(s_1, a_1, s_3) = 0.2$$
$$\mathcal{P}(s_1, a_2, s_1) = 0.1, \mathcal{P}(s_1, a_2, s_2) = 0.2, \mathcal{P}(s_1, a_2, s_3) = 0.7$$
$$\mathcal{P}(s_2, a_1, s_1) = 0.3, \mathcal{P}(s_2, a_1, s_2) = 0.3, \mathcal{P}(s_2, a_1, s_3) = 0.4$$
$$\mathcal{P}(s_2, a_2, s_1) = 0.5, \mathcal{P}(s_2, a_2, s_2) = 0.3, \mathcal{P}(s_2, a_2, s_3) = 0.2$$
The Reward Function 
$$\mathcal{R}: \mathcal{N} \times \mathcal{A} \rightarrow \mathbb{R}$$
is defined as:
$$\mathcal{R}(s_1, a_1) = 8.0, \mathcal{R}(s_1, a_2) = 10.0$$
$$\mathcal{R}(s_2, a_1) = 1.0, \mathcal{R}(s_2, a_2) = -1.0$$
Assume discount factor $\gamma = 1$.

Your task is to determine an Optimal Deterministic Policy **by manually working out** (not with code) simply the first two iterations of Value Iteration algorithm. 

- Initialize the Value Function for each state to be it's $\max$ (over actions) reward, i.e., we initialize the Value Function to be $v_0(s_1) = 10.0, v_0(s_2) = 1.0, v_0(s_3) = 0.0$. Then manually calculate $q_k(\cdot, \cdot)$ and $v_k(\cdot)$ from $v_{k - 1}( \cdot)$ using the Value Iteration update, and then calculate the greedy policy $\pi_k(\cdot)$ from $q_k(\cdot, \cdot)$ for $k = 1$ and $k = 2$ (hence, 2 iterations).
- Now argue that $\pi_k(\cdot)$ for $k > 2$ will be the same as $\pi_2(\cdot)$. Hint: You can make the argument by examining the structure of how you get $q_k(\cdot, \cdot)$ from $v_{k-1}(\cdot)$. With this argument, there is no need to go beyond the two iterations you performed above, and so you can establish $\pi_2(\cdot)$ as an Optimal Deterministic Policy for this MDP.

Answer <br>
Given the initialized value function, we apply Bellman Optimality Equations to get the value functions in the next iterations. <br>
Notice that since $s_3$ is a terminal state and with initial value of 0. It will be 0 in the follow iterations. <br>
Iteration 1: <br>
$Q^*(s_1,a_1) = 8 + 0.2*10 + 0.6*1 = 10.6, Q^*(s_1,a_2) = 10 + 0.1*10 + 0.2*1 = 11.2$ <br>
$Q^*(s_2,a_1) = 1 + 0.3*10 + 0.3*1 = 4.3, Q^*(s_2,a_2) = -1 + 0.5*10 + 0.3*1 = 4.3$ <br>
So we get $v_1(s_1) = 11.2, v_1(s_2) = 4.3, v_1(s_3) = 0$ <br>
The greedy policy is: $\pi_1(s_1) = a_2, \pi_1(s_2) = a_1,a_2$ by determining q is larger under which action <br>
Iteration 2: <br>
$Q^*(s_1,a_1) = 8 + 0.2*11.2 + 0.6*4.3 = 12.82, Q^*(s_1,a_2) = 10 + 0.1*11.2 + 0.2*4.3 = 11.98$ <br>
$Q^*(s_2,a_1) = 1 + 0.3*11.2 + 0.3*4.3 = 5.65, Q^*(s_2,a_2) = -1 + 0.5*11.2 + 0.3*4.3 = 5.89$ <br>
So we get $v_2(s_1) = 12.82, v_2(s_2) = 5.89, v_2(s_3) = 0$ <br>
Similarly, the greedy policy is: $\pi_2(s_1) = a_1, \pi_2(s_2) = a_2$ <br>
<br>
<br>
Claim: staring from the second iteration, the optimal policy in the all iterations ahead will be: $\pi_i(s_1) = a_1, \pi_i(s_2) = a_2$ <br>
Proof by induction, starting from the second iteration, $v_i(s_1) > 12, v_i(s_2) > 5$ <br>
The result of iteration 2 has prooved the basis <br>
For the inductive step, given $v_i(s_1) > 12, v_i(s_2) > 5$, $v_i(s_1) > 13.4 > 12, v_i(s_2) > 6.5 > 5$. The statement holds. <br>
Then, based on that statment, we compare Q under two actions. <br>
$Q^*(s_1,a_1) - Q^*(s_1,a_2) = -2 + 0.1*v_i(s_1) + 0.4*v_i(s_2) >0, Q^*(s_2,a_2) - Q^*(s_2,a_1) = -2 + 0.2v_i(s_1) > 0$.
Thus, we proved the claim:  the optimal policy in the all iterations ahead will be: $\pi_i(s_1) = a_1, \pi_i(s_2) = a_2$ for $i \geq 2$

## Question 3

**Job-Hopping and Wages-Utility-Maximization.** 
You are a worker who starts every day either employed or unemployed. If you start your day employed, you work on your job for the day (one of $n$ jobs, as elaborated later) and you get to earn the wage of the job for the day. However, at the end of the day, you could lose your job with probability $\alpha \in [0,1]$, in which case you start the next day unemployed. If at the end of the day, you do not lose your job (with probability $1-\alpha$), then you will start the next day with the same job (and hence, the same daily wage). On the other hand, if you start your day unemployed, then you will be randomly offered one of $n$ jobs with daily wages $w_1, w_2, \ldots w_n \in \mathbb{R}^+$ with respective job-offer probabilities $p_1, p_2, \ldots p_n \in [0,1]$ (with $\sum_{i=1}^n p_i = 1$). You can choose to either accept or decline the offered job. If you accept the job-offer, your day progresses exactly like the **employed-day** described above (earning the day's job wage and possibly (with probability $\alpha$) losing the job at the end of the day). However, if you decline the job-offer, you spend the day unemployed, receive the unemployment wage $w_0 \in \mathbb{R}^+$ for the day, and start the next day unemployed. The problem is to identify the optimal choice of accepting or rejecting any of the job-offers the worker receives, in a manner that maximizes the infinite-horizon **Expected Discounted-Sum of Wages Utility**. Assume the daily discount factor for wages (employed or unemployed) is $\gamma \in [0,1)$. Assume Wages Utility function to be $U(w) = \log(w)$ for any wage amount $w \in \mathbb{R}^+$. So you are looking to maximize
$$\mathbb{E}[\sum_{u=t}^\infty \gamma^{u-t} \cdot \log(w_{i_u})]$$
at the start of a given day $t$ ($w_{i_u}$ is the wage earned on day $u$, $0\leq i_u \leq n$ for all $u\geq t$).

- Express with clear mathematical notation the state space, action space, transition function, reward function, and write the Bellman Optimality Equation customized for this MDP.
- You can solve this Bellman Optimality Equation (hence, solve for the Optimal Value Function and the Optimal Policy) with a numerical iterative algorithm (essentially a Dynamic Programming algorithm customized to this problem). Write Python code for this numerical algorithm. Clearly define the inputs and outputs of your algorithm with their types (int, float, List, Mapping etc.). For this problem, don't use any of the MDP/DP code from the git repo, write this customized algorithm from scratch.

We have those states: $[s_1, s_2, ...,s_n]*[U, E]$ meaning offered in ith firm and currently employed, unemployed. And the action $[r, d]$ (accept, decline) <br> 
Thus, we will have those expressions for transition probabilities and expected rewards <br>
$p(s_iE, r, s_kU) = \alpha p_k, p(s_iU, r, s_kU) = \alpha p_k$ meaning you get unemployed in i and offered in k <br> 
$p(s_iE, r, s_iE) = 1 - \alpha, p(s_iU, r, s_iE) = 1 - \alpha$ meaning you accpet and keep your job <br> 
$p(s_iU, d, s_kU) = p_k$ meaning unemployed, offered in k, decline <br> 
Expected Reward $R(s_iU, d) = U(w_0), R(s_iU, a) = U(w_1), R(s_iE, a) = U(w_1)$ <br>
The fundamental trick for that question is: no matter you are the one who keeps your job to the second day, or the one who unemployed on the second day but offered in the same firm and accept, you will have a exactly same workday, with same wages and same probability to be sacked. <br>
<br>
Thus,  $V^*(s_iU) = max(U(w_0) + \gamma \sum_{k=1}^n p_k V^*(s_kU), V^*(s_iE))$ <br>
The first term is for decline, the second term is for accept. <br>
$V^*(s_iE) = U(w_i) + \gamma (\alpha \sum_{k=1}^n p_k V^*(s_kU) + (1-\alpha) V^*(s_iE))$ <br>
We can use previous t results. Denote $V_i = V^*(s_iE)$, we will accept the offer if <br>
$V_i > U(w_0) + \gamma \sum_{k=1}^n p_k V^*(s_kU)$

In [ ]:
class JobHoppingMP:

    ps: Sequence[float]
    wages: Sequence[float]
    gamma: float
    alpha: float

    def compute_utility(self) -> Sequence[float]:
        return [np.log(x) for x in self.wages]

    def get_opt_vf(self, tol: float = 1e-8) -> Sequence[float]:
        n = len(self.ps)
        U = self.compute_utility()
        vs = [0.0] * (n + 1)
        epsilon = 1
        while epsilon >= tol:
            vs_old = vs.deepcopy()
            v_cache = 0.0
            for i in range(1, n + 1):
                v_cache += self.ps[i] * max(vs[i], U[0] + self.gamma * vs[0])
            vs[0] = v_cache
            for i in range(1, n + 1):
                vs[i] = U[i] + self.gamma * (self.alpha * vs[0] + (1 - self.alpha) * vs[i])
            epsilon = max(abs(np.asarray(vs_old) - np.asarray(vs)))
        return vs

    def get_opt_policy(self) -> Sequence[str]:
        n = len(self.ps)
        U = self.compute_utility()
        vs = self.get_opt_vf()
        return ["A" if vs[i] > U[0] + self.gamma * vs[0]
                else "D" for i in range(1, n + 1)]
    


## Question 4 (Do Not Grade)

**Two-Stores Inventory Control.** 
We extend the capacity-constrained inventory example implemented in [rl/chapter3/simple_inventory_mdp_cap.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter3/simple_inventory_mdp_cap.py) as a `FiniteMarkovDecisionProcess` (the Finite MDP model for the capacity-constrained inventory example is described in detail in Chapters 1 and 2 of the RLForFinanceBook). Here we assume that we have two different stores, each with their own separate capacities $C_1$ and $C_2$, their own separate Poisson probability distributions of demand (with means $\lambda_1$ and $\lambda_2$), their own separate holding costs $h_1$ and $h_2$, and their own separate stockout costs $p_1$ and $p_2$. At 6pm upon stores closing each evening, each store can choose to order inventory from a common supplier (as usual, ordered inventory will arrive at the store 36 hours later). We are also allowed to transfer inventory from one store to another, and any such transfer happens overnight, i.e., will arrive by 6am next morning (since the stores are fairly close to each other). Note that the orders are constrained such that following the orders on each evening, each store's inventory position (sum of on-hand inventory and on-order inventory) cannot exceed the store's capacity (this means the action space is constrained to be finite). Each order made to the supplier incurs a fixed transportation cost of $K_1$ (fixed-cost means the cost is the same no matter how many units of non-zero inventory a particular store orders). Moving any non-zero inventory between the two stores incurs a fixed transportation cost of $K_2$. 

Model this as a derived class of `FiniteMarkovDecisionProcess` much like we did for `SimpleInventoryMDPCap` in the code repo. Set up instances of this derived class for different choices of the problem parameters (capacities, costs etc.), and determine the Optimal Value Function and Optimal Policy by invoking the function `value_iteration` (or `policy_iteration`) from file [rl/dynamic_programming.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/dynamic_programming.py).

Analyze the obtained Optimal Policy and verify that it makes intuitive sense as a function of the problem parameters.

In [ ]:
InventoryState = Tuple[int, int]
InventoryAction = Tuple[int, int]

InvOrderMapping = Mapping[
    InventoryState,
    Mapping[InventoryAction, Categorical[Tuple[InventoryState, float]]]
]


class TwoStoresInventoryMDP(
        FiniteMarkovDecisionProcess[InventoryState, InventoryAction]
):

    def __init__(
        self,
        store1_capacity: int,
        store2_capacity: int,
        store1_poisson_lambda: float,
        store2_poisson_lambda: float,
        store1_holding_cost: float,
        store2_holding_cost: float,
        store1_stockout_cost: float,
        store2_stockout_cost: float,
        small_truck_cost: float,
        big_truck_cost: float
    ):
        self.c1: int = store1_capacity
        self.c2: int = store2_capacity
        self.lambda1: float = store1_poisson_lambda
        self.lambda2: float = store2_poisson_lambda
        self.h1: float = store1_holding_cost
        self.h2: float = store2_holding_cost
        self.p1: float = store1_stockout_cost
        self.p2: float = store2_stockout_cost
        self.k1: float = small_truck_cost
        self.k2: float = big_truck_cost

        self.f1 = poisson(store1_poisson_lambda)
        self.f2 = poisson(store2_poisson_lambda)
        super().__init__(self.get_action_transition_reward_map())

    def actions_for_state(
        self,
        state: InventoryState
    ) -> Sequence[InventoryAction]:
        inv1, inv2 = state

        return [(a1, a2) for a1 in range(max(-(self.c1 - inv1), -inv2), min(self.c2 - inv2, inv1) + 1) \
                for a2 in range(-(self.c2-(inv1 - a1)), self.c2-(inv2 + a1) + 1)]

    def get_action_transition_reward_map(self) -> InvOrderMapping:
        d: Dict[InventoryState, Dict[InventoryAction, Categorical[Tuple[
            InventoryState, float]]]] = {}

        for inv1 in range(self.c1 + 1):
            for inv2 in range(self.c2 + 1):
                h1_cost: float = self.h1 * inv1
                h2_cost: float = self.h2 * inv2
                d1: Dict[InventoryAction, Categorical[Tuple[InventoryState, float]]] = {}

                for a1, a2 in self.actions_for_state((inv1, inv2)):
                    store1_morning: int =  inv1 - a1 - min(a2, 0)
                    store2_morning: int =  inv2 + a1 + max(a2, 0)

                    sr_probs_dict: Dict[Tuple[InventoryState, float], float] = {}
                    
                    for demand1 in range(store1_morning + 1):
                        for demand2 in range(store2_morning + 1):
                            next_state = (store1_morning - demand1, store2_morning - demand2)
                            prob = (self.f1.pmf(demand1) * 
                                    self.f2.pmf(demand2))
                            
                            # Implementing logic for Cases I, II, III, IV
                            if next_state[0] > 0 and next_state[1] > 0:
                                # Case I
                                reward = - (h1_cost + h2_cost + 
                                            self.k1 * (a1 != 0) + 
                                            self.k2 * (a2 != 0))
                            elif next_state[1] == 0 and next_state[0] > 0: # state I > 0, State II == 0
                                # Case II
                                reward = - (h2_cost + h1_cost + self.k1 * (a1 != 0) +
                                            self.k2 * (a2 != 0))
                                stockout_cost = sum(self.f2.pmf(d) * 
                                                    (d - store2_morning) * self.p2
                                                    for d in range(store2_morning, int(self.f2.ppf(0.9999)) + 1))
                                reward -= stockout_cost
                            elif next_state[1] > 0 and next_state[0] == 0:
                                # Case III
                                reward = - (h1_cost + h2_cost + self.k1 * (a1 != 0) +
                                            self.k2 * (a2 != 0))
                                stockout_cost = sum(self.f1.pmf(d) * 
                                                    (d - store1_morning) * self.p1
                                                    for d in range(store1_morning, int(self.f1.ppf(0.9999)) + 1))
                                reward -= stockout_cost
                            else:
                                # Case IV
                                reward = - (h1_cost + h2_cost + self.k1 * (a1 != 0) +
                                            self.k2 * (a2 != 0))
                                stockout_cost_1 = sum(self.f1.pmf(d) * 
                                                      (d - store1_morning) * self.p1
                                                      for d in range(store1_morning, int(self.f1.ppf(0.9999)) + 1))
                                stockout_cost_2 = sum(self.f2.pmf(d) * 
                                                      (d - store2_morning) * self.p2
                                                      for d in range(store2_morning, int(self.f2.ppf(0.9999)) + 1))
                                reward -= (stockout_cost_1 + stockout_cost_2)

                            sr_probs_dict[next_state, reward] = prob

                    d1[(a1, a2)] = Categorical(sr_probs_dict)

                d[(inv1, inv2)] = d1
        return d
